In [1]:
import findspark
findspark.init("c:\spark")
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.master("local[4]") \
.appName("DataExplore") \
.config("spark.executor.memory","4g") \
.config("spark.driver.memory","2g") \
.getOrCreate()

## Veri Setini Okuma Öncesi Kontroller

### 1. Veri setine ulaşma:
1. adult.data ve adult.test adında iki veri dosyası var.
2. Notepad++ ile içine bakalım. virgülle ayrılmış ve başlıkları olan bir dosya olduğunu gördük.

In [23]:
adult_train_df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",",") \
.csv("/C:/Program Files/data_sets_pf/adult.data")

In [24]:
adult_test_df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",",") \
.csv("/C:/Program Files/data_sets_pf/adult.test")

In [31]:
adult_train_df.show(5)

+---+-----------------+--------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
|age|        workclass|  fnlwgt| education|education_num|     marital_status|        occupation|  relationship|  race|    sex|capital_gain|capital_loss|hours_per_week|native_country|output|
+---+-----------------+--------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
| 39|        State-gov| 77516.0| Bachelors|         13.0|      Never-married|      Adm-clerical| Not-in-family| White|   Male|      2174.0|         0.0|          40.0| United-States| <=50K|
| 50| Self-emp-not-inc| 83311.0| Bachelors|         13.0| Married-civ-spouse|   Exec-managerial|       Husband| White|   Male|         0.0|         0.0|          13.0| United-States| <=50K|
| 38|          Private|215646.0|   HS-grad|       

In [27]:
import pandas as pd

adult_train_df.toPandas().head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,output
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


In [32]:
adult_test_df.toPandas().head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,output
0,25,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K.
1,38,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K.
2,28,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K.
3,44,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K.
4,18,?,103497.0,Some-college,10.0,Never-married,?,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K.


Veri keşfinde 2 ayrı veriseti istemiyoruz çünkü genel olarka verilerde düzenleme yapaıcaz, bu yüzden verisetlerini birleştiriyoruz

In [34]:
adult_whole_df = adult_train_df.union(adult_test_df)

In [35]:
adult_whole_df.toPandas().head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,output
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


kayıp veri var mı diye kontrol

In [39]:
print(adult_train_df.count())
print(adult_test_df.count())
print(adult_whole_df.count())

32561
16281
48842


## Veri Seti İle Şema Karşılaştırılması

In [40]:
adult_whole_df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: double (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: double (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: double (nullable = true)
 |-- capital_loss: double (nullable = true)
 |-- hours_per_week: double (nullable = true)
 |-- native_country: string (nullable = true)
 |-- output: string (nullable = true)



## Nümerik Değişkenlerin İstatistikleri

In [42]:
adult_whole_df.describe(["age","fnlwgt","education_num","capital_gain","capital_loss","hours_per_week"]).toPandas().head()

,summary,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
0,count,48842,48842,48842,48842,48842,48842
1,mean,38.64358543876172,189664.13459727284,10.078088530363212,1079.0676262233324,87.50231358257237,40.422382375824085
2,stddev,13.710509934443563,105604.02542315733,2.570972755592263,7452.019057655401,403.0045521243599,12.3914440242523
3,min,17,12285.0,1.0,0.0,0.0,1.0
4,max,90,1490400.0,16.0,99999.0,4356.0,99.0


## Kategorik Değişkenlerin İncelenmesi

### 1. workclass

In [45]:
from pyspark.sql import functions as F
adult_whole_df.groupBy(F.col("workclass")).agg({"*":"count"}) \
.toPandas().head()

,workclass,count(1)
0,State-gov,1981
1,Federal-gov,1432
2,Self-emp-not-inc,3862
3,Local-gov,3136
4,Private,33906


In [59]:
adult_whole_df.createOrReplaceTempView("table1")
spark.sql("select workclass,count(*) from table1 group by workclass").toPandas()

,workclass,count(1)
0,State-gov,1981
1,Federal-gov,1432
2,Self-emp-not-inc,3862
3,Local-gov,3136
4,Private,33906
5,?,2799
6,Self-emp-inc,1695
7,Without-pay,21
8,Never-worked,10


-> 2799 tane ? var bu nedir. Muhtemelen kayıp bilgiler
-> Without-pay ve Never-worked sınıfları çok az kullanılmış, çıkarılabilir.

## 2. Education 

In [67]:
adult_whole_df.groupBy(F.col("education")).agg({"*":"count"}).toPandas()

,education,count(1)
0,Prof-school,834
1,10th,1389
2,7th-8th,955
3,5th-6th,509
4,Assoc-acdm,1601
5,Assoc-voc,2061
6,Masters,2657
7,12th,657
8,Preschool,83
9,9th,756


In [65]:
spark.sql("select education, count(*) from table1 group by education").toPandas()

,education,count(1)
0,Prof-school,834
1,10th,1389
2,7th-8th,955
3,5th-6th,509
4,Assoc-acdm,1601
5,Assoc-voc,2061
6,Masters,2657
7,12th,657
8,Preschool,83
9,9th,756



Yorum: Genel bir sıkıntı görünmüyor ancak çok fazla kategori var belki bazıları birleştirilebilir.

      1st-4th, 5th-6th, 7th-8th: elementary-school

      9th, 10th, 11th, 12th: high-school

      Masters, Doctorate: high-education

      Bachelors, Some-college: undergraduate

## 3. marital_status

In [68]:
adult_whole_df.groupBy(F.col("marital_status")).agg({"*":"count"}).toPandas()

,marital_status,count(1)
0,Widowed,1518
1,Married-spouse-absent,628
2,Married-AF-spouse,37
3,Married-civ-spouse,22379
4,Divorced,6633
5,Never-married,16117
6,Separated,1530


bir sorun gözükmüyor

## 4. occupation

In [69]:
adult_whole_df.groupBy(F.col("occupation")).agg({"*":"count"}).toPandas()

,occupation,count(1)
0,Farming-fishing,1490
1,Handlers-cleaners,2072
2,Prof-specialty,6172
3,Adm-clerical,5611
4,Exec-managerial,6086
5,Craft-repair,6112
6,Sales,5504
7,?,2809
8,Tech-support,1446
9,Transport-moving,2355


-> 2809 tane ? var. Bunlar muhtemelen bilinmeyenler. Ayrıca Armed-Forces 15 kişi. Bu sınıfa ait kayıtlar çıkarılabilir.

## 5.relationship

In [72]:
adult_whole_df.groupBy(F.col("relationship")).agg({"*":"count"}).toPandas()

,relationship,count(1)
0,Husband,19716
1,Own-child,7581
2,Not-in-family,12583
3,Other-relative,1506
4,Wife,2331
5,Unmarried,5125


sorun yok

## 6.race

In [73]:
adult_whole_df.groupBy(F.col("race")) \
.agg({"*":"count"}) \
.toPandas()

,race,count(1)
0,Asian-Pac-Islander,1519
1,Black,4685
2,Other,406
3,White,41762
4,Amer-Indian-Eskimo,470


## 7.sex

In [74]:
adult_whole_df.groupBy(F.col("sex")) \
.agg({"*":"count"}) \
.toPandas()

,sex,count(1)
0,Male,32650
1,Female,16192


## 8.native_country

In [87]:
adult_whole_df.groupBy(F.col("native_country")) \
.agg({"*":"count"}).orderBy(F.desc("count(1)")) \
.toPandas()

,native_country,count(1)
0,United-States,43832
1,Mexico,951
2,?,857
3,Philippines,295
4,Germany,206
5,Puerto-Rico,184
6,Canada,182
7,El-Salvador,155
8,India,151
9,Cuba,138


Çoğunluk US

## 9. output

In [76]:
adult_whole_df.groupBy(F.col("output")) \
.agg({"*":"count"}) \
.toPandas()

,output,count(1)
0,>50K,7841
1,>50K.,3846
2,<=50K.,12435
3,<=50K,24720


50k dan büüyk ve küçük olmalı diğer '.' içeren değişkenlerden nokta çıkartılmalı

/////////////////////// VERİ TEMİZLİĞİ İÇİN TAVSİYELER ////////////////////////////////////////////////

  1. Tüm sütunları boşluk kontrolü yap.
  2. ? içeren workclass, occupation  var bunların ? içerdiği satırlar tekrar incelenmeli.
      ? işaretleri sistematik bir şekilde mi oluşmuş yoksa bu oluşum tesadüfi mi?
      ? kayıtlarının oluşması altında yatan bir mekanizma var mı?
      Bu sistematik hata yakalanırsa veri doldurma (imputation) yoksa satır silme yapılsın.
  3. workclass niteliğinde never-worked ve without-pay sınıfları ve
      occupation niteliğinde  Armed-Forces  sınıfı
    çok az tekrarlanmış. Veri setinden çıkarılabilir.
  4. education niteliğindeki:
          1st-4th, 5th-6th, 7th-8th: elementary-school
          9th, 10th, 11th, 12th: high-school
          Masters, Doctorate: high-education
          Bachelors, Some-college: undergraduate
     sınıfları yukarıdaki gibi birleştirilebilir.
  5. output (hedef değişkendeki) >50K. ve <=50K. değerlerindeki "." kaldırılmalı